In [ ]:
import pygame
import random
import math

pygame.init()

# ============================================
# คลาส LavaPit - บ่อลาวา (อุปสรรค)
# ============================================
class LavaPit:
    """บ่อลาวาที่จะทำให้สูญเสียคน"""
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.width = random.randint(60, 120)
        self.height = random.randint(40, 80)
        self.damage = random.randint(5, 15)
        self.active = True
        self.color = (255, 165, 0)  # ORANGE
        
    def draw(self):
        """วาดบ่อลาวา"""
        if self.active:
            offset_x = random.randint(-2, 2)
            offset_y = random.randint(-2, 2)
            pygame.draw.ellipse(screen, self.color, 
                              (int(self.x - self.width//2 + offset_x), 
                               int(self.y - self.height//2 + offset_y), 
                               self.width, self.height))
            pygame.draw.ellipse(screen, RED, 
                              (int(self.x - self.width//2 + offset_x + 5), 
                               int(self.y - self.height//2 + offset_y + 5), 
                               self.width - 10, self.height - 10))
    
    def check_collision(self, crowd):
        """เช็คชนกับฝูงชน"""
        if not self.active or len(crowd.people) == 0:
            return False
        
        hit = False
        for person in crowd.people[:]:
            dist_x = abs(person.x - self.x)
            dist_y = abs(person.y - self.y)
            
            if dist_x < self.width//2 and dist_y < self.height//2:
                hit = True
                break
        
        if hit:
            crowd.remove_people(min(self.damage, crowd.count))
            self.active = False
            return True
        return False


# ============================================
# ตั้งค่าเกม
# ============================================
WIDTH, HEIGHT = 800, 700
FPS = 60

# ความเร็ว
SCROLL_SPEED = 5
PLAYER_SPEED = 14
BULLET_SPEED = 15
ENEMY_SPEED = 2

# ขอบเขตถนน
ROAD_LEFT = 50
ROAD_RIGHT = WIDTH - 50
ROAD_WIDTH = ROAD_RIGHT - ROAD_LEFT

# สี
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (100, 150, 255)
CYAN = (0, 200, 255)
RED = (255, 50, 50)
GREEN = (50, 255, 100)
YELLOW = (255, 255, 100)
ORANGE = (255, 165, 0)
GRAY = (40, 40, 40)
DARK_GRAY = (30, 30, 30)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Math Crowd Runner")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 48)
small_font = pygame.font.Font(None, 28)
big_font = pygame.font.Font(None, 72)
huge_font = pygame.font.Font(None, 36)

# โหลดรูปภาพ
try:
    ai_hoshino_img = pygame.image.load('ai_hoshino.jpg')
    ai_hoshino_img = pygame.transform.scale(ai_hoshino_img, (30, 30))
except:
    ai_hoshino_img = None

try:
    knife_img = pygame.image.load('knife.png')
    knife_img = pygame.transform.scale(knife_img, (20, 20))
except:
    knife_img = None

try:
    stage_bg = pygame.image.load('stage.png')
    stage_bg = pygame.transform.scale(stage_bg, (WIDTH, HEIGHT))
except:
    stage_bg = None

try:
    ai_dead_img = pygame.image.load('ai_dead.jpg')
    ai_dead_img = pygame.transform.scale(ai_dead_img, (WIDTH, HEIGHT))  # เต็มจอ
except:
    ai_dead_img = None

try:
    ai_win_img = pygame.image.load('ai_win.png')
    ai_win_img = pygame.transform.scale(ai_win_img, (WIDTH, HEIGHT))
except:
    ai_win_img = None

# โหลดเพลง
try:
    pygame.mixer.music.load('bg_song.mp3')
except:
    pass

# Game State
game_state = "MENU"
current_level = 1


# ============================================
# คลาส Button
# ============================================
class Button:
    def __init__(self, x, y, width, height, text, color):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.color = color
        self.hover = False
        
    def draw(self):
        color = self.color if not self.hover else tuple(min(c + 30, 255) for c in self.color)
        pygame.draw.rect(screen, color, self.rect)
        pygame.draw.rect(screen, WHITE, self.rect, 3)
        
        text_surf = font.render(self.text, True, WHITE)
        text_rect = text_surf.get_rect(center=self.rect.center)
        screen.blit(text_surf, text_rect)
    
    def check_click(self, pos):
        self.hover = self.rect.collidepoint(pos)
        return self.hover


# ============================================
# คลาส Person - คนในทีมเรา
# ============================================
class Person:
    """คนหนึ่งคนในทีม มีปืน"""
    def __init__(self, x, y, crowd_size=10):
        self.x = x
        self.y = y
        self.base_size = 10
        self.crowd_size = crowd_size
        self.update_size()
        self.color = BLUE
        self.has_gun = True
        self.shoot_cooldown = 0
        self.shoot_rate = 20
    
    def update_size(self):
        """ปรับขนาดตามจำนวนคนในทีม"""
        if self.crowd_size <= 10:
            self.size = 15
        elif self.crowd_size <= 50:
            self.size = 12
        elif self.crowd_size <= 100:
            self.size = 10
        else:
            self.size = max(8, 15 - int(self.crowd_size / 50))
        
    def update(self, target_x, target_y):
        """เคลื่อนที่ไปหาตำแหน่งในฟอร์เมชั่น"""
        dx = target_x - self.x
        dy = target_y - self.y
        dist = math.sqrt(dx*dx + dy*dy)
        
        if dist > 2:
            speed = 4
            self.x += (dx / dist) * speed
            self.y += (dy / dist) * speed
        
        margin = self.size + 5
        self.x = max(ROAD_LEFT + margin, min(ROAD_RIGHT - margin, self.x))
        self.y = max(50, min(HEIGHT - 50, self.y))
        
        if self.shoot_cooldown > 0:
            self.shoot_cooldown -= 1
    
    def can_shoot(self):
        return self.has_gun and self.shoot_cooldown == 0
    
    def shoot(self, target):
        if self.can_shoot():
            self.shoot_cooldown = self.shoot_rate
            return Bullet(self.x, self.y, target)
        return None
    
    def draw(self):
        if ai_hoshino_img:
            scaled_img = pygame.transform.scale(ai_hoshino_img, (self.size * 2, self.size * 2))
            img_rect = scaled_img.get_rect(center=(int(self.x), int(self.y)))
            screen.blit(scaled_img, img_rect)
        else:
            pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), self.size)
            if self.has_gun:
                gun_length = max(8, self.size + 2)
                pygame.draw.line(screen, WHITE, 
                               (int(self.x), int(self.y)), 
                               (int(self.x), int(self.y - gun_length)), 2)


# ============================================
# คลาส Crowd - กลุ่มคนทีมเรา
# ============================================
class Crowd:
    def __init__(self):
        self.people = [Person(WIDTH // 2, HEIGHT - 100, 1)]
        self.center_x = WIDTH // 2
        self.count = 1
        
    def add_people(self, amount):
        if amount > 0:
            current = len(self.people)
            amount = min(amount, 500 - current)
            
            for _ in range(amount):
                angle = random.uniform(0, math.pi * 2)
                radius = random.uniform(10, 30)
                x = self.center_x + math.cos(angle) * radius
                y = HEIGHT - 100 + math.sin(angle) * radius
                self.people.append(Person(x, y, len(self.people) + 1))
            self.count = len(self.people)
            self.update_all_sizes()
    
    def remove_people(self, amount):
        if amount > 0 and len(self.people) > 0:
            amount = min(amount, len(self.people))
            self.people = self.people[:-amount]
            self.count = len(self.people)
            if self.count > 0:
                self.update_all_sizes()
    
    def update_all_sizes(self):
        for person in self.people:
            person.crowd_size = self.count
            person.update_size()
    
    def multiply_people(self, multiplier):
        if multiplier > 0:
            current = len(self.people)
            to_add = current * (multiplier - 1)
            to_add = min(to_add, 500 - current)
            if to_add > 0:
                self.add_people(to_add)
    
    def divide_people(self, divisor):
        if divisor > 0 and self.count > 0:
            new_count = max(1, self.count // divisor)
            to_remove = self.count - new_count
            if to_remove > 0:
                self.remove_people(to_remove)
    
    def power_people(self, power):
        current = len(self.people)
        if current > 0:
            new_count = min(500, int(current ** power))
            if new_count > current:
                self.add_people(new_count - current)
            elif new_count < current:
                self.remove_people(current - new_count)
    
    def sqrt_people(self):
        current = len(self.people)
        if current > 0:
            new_count = max(1, int(math.sqrt(current)))
            if new_count < current:
                self.remove_people(current - new_count)
    
    def move(self, dx):
        new_x = self.center_x + dx
        margin = 60
        self.center_x = max(ROAD_LEFT + margin, min(ROAD_RIGHT - margin, new_x))
    
    def update(self):
        num = len(self.people)
        if num == 0:
            return
        
        max_per_ring = 12
        person_index = 0
        ring = 0
        
        while person_index < num:
            people_in_ring = min(max_per_ring + ring * 3, num - person_index)
            radius = 18 + ring * 22
            
            for i in range(people_in_ring):
                if person_index >= num:
                    break
                
                angle = (i / people_in_ring) * math.pi * 2 + (ring * 0.5)
                target_x = self.center_x + math.cos(angle) * radius
                target_y = HEIGHT - 100 + math.sin(angle) * radius * 0.4
                
                self.people[person_index].update(target_x, target_y)
                person_index += 1
            
            ring += 1
    
    def try_shoot(self, enemies):
        bullets = []
        for person in self.people:
            closest_enemy = None
            min_dist = float('inf')
            
            for enemy in enemies:
                if not enemy.alive or not enemy.active:
                    continue
                dist = math.sqrt((enemy.x - person.x)**2 + (enemy.y - person.y)**2)
                if dist < min_dist and dist < 350:
                    min_dist = dist
                    closest_enemy = enemy
            
            if closest_enemy and person.can_shoot():
                bullet = person.shoot(closest_enemy)
                if bullet:
                    bullets.append(bullet)
        
        return bullets
    
    def draw(self):
        for person in self.people:
            person.draw()


# ============================================
# คลาส Gate - ประตูเลือก
# ============================================
class Gate:
    def __init__(self, y, level):
        self.y = y
        self.height = 150
        self.used = False
        self.level = level
        
        if level == 1:
            left_num = random.randint(-50, 50)
            right_num = random.randint(-50, 50)
            
            if left_num >= 0:
                self.left_op = (f'+{left_num}', 'add', left_num)
            else:
                self.left_op = (f'{left_num}', 'subtract', abs(left_num))
            
            if right_num >= 0:
                self.right_op = (f'+{right_num}', 'add', right_num)
            else:
                self.right_op = (f'{right_num}', 'subtract', abs(right_num))
                
        elif level == 2:
            ops = [
                ('x2', 'multiply', 2), ('x3', 'multiply', 3),
                ('x4', 'multiply', 4), ('x5', 'multiply', 5),
                ('÷2', 'divide', 2), ('÷3', 'divide', 3),
                ('÷4', 'divide', 4), ('÷5', 'divide', 5),
                ('÷6', 'divide', 6), ('÷7', 'divide', 7),
                ('÷8', 'divide', 8), ('÷10', 'divide', 10),
                ('√', 'sqrt', 0), ('^2', 'power', 2), ('^3', 'power', 3),
            ]
            if random.random() < 0.7:
                self.left_op = random.choice(ops)
            else:
                left_rand = random.randint(-30, 30)
                if left_rand >= 0:
                    self.left_op = (f'+{left_rand}', 'add', left_rand)
                else:
                    self.left_op = (f'{left_rand}', 'subtract', abs(left_rand))
            
            if random.random() < 0.7:
                self.right_op = random.choice(ops)
            else:
                right_rand = random.randint(-30, 30)
                if right_rand >= 0:
                    self.right_op = (f'+{right_rand}', 'add', right_rand)
                else:
                    self.right_op = (f'{right_rand}', 'subtract', abs(right_rand))
        else:
            ops = [
                ('x2', 'multiply', 2), ('x3', 'multiply', 3),
                ('x4', 'multiply', 4), ('x5', 'multiply', 5),
                ('÷2', 'divide', 2), ('÷3', 'divide', 3),
                ('÷5', 'divide', 5), ('÷7', 'divide', 7),
                ('÷10', 'divide', 10), ('÷15', 'divide', 15),
                ('÷20', 'divide', 20), ('√', 'sqrt', 0),
                ('^2', 'power', 2), ('^3', 'power', 3),
            ]
            if random.random() < 0.8:
                self.left_op = random.choice(ops)
            else:
                left_rand = random.randint(-40, 40)
                if left_rand >= 0:
                    self.left_op = (f'+{left_rand}', 'add', left_rand)
                else:
                    self.left_op = (f'{left_rand}', 'subtract', abs(left_rand))
            
            if random.random() < 0.8:
                self.right_op = random.choice(ops)
            else:
                right_rand = random.randint(-40, 40)
                if right_rand >= 0:
                    self.right_op = (f'+{right_rand}', 'add', right_rand)
                else:
                    self.right_op = (f'{right_rand}', 'subtract', abs(right_rand))
        
        self.split_x = WIDTH // 2
    
    def check_collision(self, crowd):
        if self.used or len(crowd.people) == 0:
            return False
        
        if abs(crowd.people[0].y - self.y) < 75:
            if crowd.center_x < self.split_x:
                self.apply_operation(crowd, self.left_op)
                self.used = True
                return True
            else:
                self.apply_operation(crowd, self.right_op)
                self.used = True
                return True
        return False
    
    def apply_operation(self, crowd, operation):
        op_type = operation[1]
        value = operation[2]
        
        if op_type == 'multiply':
            crowd.multiply_people(value)
        elif op_type == 'add':
            crowd.add_people(value)
        elif op_type == 'subtract':
            crowd.remove_people(value)
        elif op_type == 'divide':
            crowd.divide_people(value)
        elif op_type == 'power':
            crowd.power_people(value)
        elif op_type == 'sqrt':
            crowd.sqrt_people()
    
    def draw(self):
        if not self.used:
            gate_color = CYAN
            
            left_rect = pygame.Rect(ROAD_LEFT, int(self.y), 
                                   (ROAD_WIDTH // 2) - 8, self.height)
            pygame.draw.rect(screen, gate_color, left_rect, 5)
            pygame.draw.rect(screen, gate_color, left_rect, 0)
            left_rect_inner = left_rect.inflate(-10, -10)
            pygame.draw.rect(screen, BLACK, left_rect_inner, 0)
            
            text = big_font.render(self.left_op[0], True, WHITE)
            text_rect = text.get_rect(center=left_rect.center)
            screen.blit(text, text_rect)
            
            right_rect = pygame.Rect(self.split_x + 8, int(self.y), 
                                    (ROAD_WIDTH // 2) - 8, self.height)
            pygame.draw.rect(screen, gate_color, right_rect, 5)
            pygame.draw.rect(screen, gate_color, right_rect, 0)
            right_rect_inner = right_rect.inflate(-10, -10)
            pygame.draw.rect(screen, BLACK, right_rect_inner, 0)
            
            text = big_font.render(self.right_op[0], True, WHITE)
            text_rect = text.get_rect(center=right_rect.center)
            screen.blit(text, text_rect)
            
            pygame.draw.rect(screen, WHITE, 
                           (self.split_x - 8, int(self.y), 16, self.height))


# ============================================
# คลาส Enemy - ศัตรู
# ============================================
class Enemy:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.size = 10
        self.color = RED
        self.hp = 1
        self.speed = ENEMY_SPEED
        self.alive = True
        self.active = False
        
    def activate(self):
        self.active = True
        
    def update(self, crowd):
        if not self.alive or not self.active:
            return
        
        if len(crowd.people) > 0:
            target = min(crowd.people, key=lambda p: 
                        math.sqrt((p.x - self.x)**2 + (p.y - self.y)**2))
            
            dx = target.x - self.x
            dy = target.y - self.y
            dist = math.sqrt(dx*dx + dy*dy)
            
            if dist > 1:
                self.x += (dx / dist) * self.speed
                self.y += (dy / dist) * self.speed
            
            if dist < 18:
                crowd.remove_people(1)
                self.alive = False
    
    def take_damage(self):
        self.hp -= 1
        if self.hp <= 0:
            self.alive = False
    
    def draw(self):
        if self.alive:
            if knife_img:
                scaled_knife = pygame.transform.scale(knife_img, (self.size * 2, self.size * 2))
                knife_rect = scaled_knife.get_rect(center=(int(self.x), int(self.y)))
                screen.blit(scaled_knife, knife_rect)
            else:
                pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), self.size)
                pygame.draw.line(screen, WHITE, 
                               (int(self.x - 8), int(self.y)), 
                               (int(self.x + 8), int(self.y)), 2)


# ============================================
# คลาส Bullet - กระสุน
# ============================================
class Bullet:
    def __init__(self, x, y, target):
        self.x = x
        self.y = y
        self.speed = BULLET_SPEED
        self.active = True
        self.target = target
        
    def update(self):
        if not self.active:
            return
            
        if not self.target or not self.target.alive:
            self.active = False
            return
        
        dx = self.target.x - self.x
        dy = self.target.y - self.y
        dist = math.sqrt(dx*dx + dy*dy)
        
        if dist > 1:
            self.x += (dx / dist) * self.speed
            self.y += (dy / dist) * self.speed
        
        if self.y < -10 or self.y > HEIGHT + 10 or self.x < -10 or self.x > WIDTH + 10:
            self.active = False
    
    def check_hit(self, enemies):
        for enemy in enemies:
            if enemy.alive:
                dist = math.sqrt((enemy.x - self.x)**2 + (enemy.y - self.y)**2)
                if dist < 18:
                    enemy.take_damage()
                    self.active = False
                    return True
        return False
    
    def draw(self):
        if self.active:
            pygame.draw.circle(screen, YELLOW, (int(self.x), int(self.y)), 4)


# ============================================
# คลาส Game
# ============================================
class Game:
    def __init__(self, level):
        self.level = level
        self.crowd = Crowd()
        self.gates = []
        self.enemies = []
        self.bullets = []
        self.lava_pits = []
        self.distance = 0
        self.spawn_timer = 0
        self.game_over = False
        self.won = False
        self.gates_passed = 0
        self.gates_needed = 4
        
        try:
            self.game_over_image = pygame.image.load('images.png')
            self.game_over_image = pygame.transform.scale(self.game_over_image, (400, 300))
        except:
            self.game_over_image = None
        
    def spawn_gate(self):
        self.gates.append(Gate(-200, self.level))
    
    def spawn_enemies(self):
        num_enemies = random.randint(8, 15)
        y = random.randint(-600, -400)
        for _ in range(num_enemies):
            x = random.randint(ROAD_LEFT + 20, ROAD_RIGHT - 20)
            self.enemies.append(Enemy(x, y))
    
    def spawn_lava_pit(self):
        if self.level >= 2:
            x = random.randint(ROAD_LEFT + 80, ROAD_RIGHT - 80)
            y = random.randint(-500, -200)
            self.lava_pits.append(LavaPit(x, y))
    
    def start_game(self):
        """เริ่มเกมด้วย gate และ enemies ทันที"""
        self.spawn_gate()
        self.spawn_enemies()
        if self.level >= 2:
            self.spawn_lava_pit()
    
    def update(self):
        if self.game_over or self.won:
            return
        
        for gate in self.gates:
            gate.y += SCROLL_SPEED
        
        for enemy in self.enemies:
            enemy.y += SCROLL_SPEED
        
        for lava in self.lava_pits:
            lava.y += SCROLL_SPEED
        
        self.spawn_timer += 1
        if self.spawn_timer % 180 == 0:  # เร็วขึ้น จาก 250
            self.spawn_gate()
        if self.spawn_timer % 200 == 0:  # เร็วขึ้น จาก 280
            self.spawn_enemies()
        if self.level >= 2 and self.spawn_timer % 280 == 0:  # เร็วขึ้น จาก 350
            self.spawn_lava_pit()
        
        self.crowd.update()
        
        new_bullets = self.crowd.try_shoot(self.enemies)
        self.bullets.extend(new_bullets)
        
        for bullet in self.bullets:
            bullet.update()
            bullet.check_hit(self.enemies)
        
        for enemy in self.enemies:
            enemy.update(self.crowd)
        
        for lava in self.lava_pits:
            lava.check_collision(self.crowd)
        
        for gate in self.gates:
            if gate.check_collision(self.crowd):
                self.gates_passed += 1
                for enemy in self.enemies:
                    enemy.activate()
        
        self.gates = [g for g in self.gates if g.y < HEIGHT + 200]
        self.enemies = [e for e in self.enemies if e.alive and e.y < HEIGHT + 100]
        self.bullets = [b for b in self.bullets if b.active]
        self.lava_pits = [l for l in self.lava_pits if l.active and l.y < HEIGHT + 100]
        
        self.distance += SCROLL_SPEED
        
        if self.crowd.count <= 0:
            self.game_over = True
        
        if self.gates_passed >= self.gates_needed:
            self.won = True
    
    def draw(self):
        if stage_bg:
            screen.blit(stage_bg, (0, 0))
        else:
            screen.fill(BLACK)
        
        pygame.draw.rect(screen, GRAY, (ROAD_LEFT, 0, ROAD_WIDTH, HEIGHT))
        pygame.draw.line(screen, WHITE, (ROAD_LEFT, 0), (ROAD_LEFT, HEIGHT), 5)
        pygame.draw.line(screen, WHITE, (ROAD_RIGHT, 0), (ROAD_RIGHT, HEIGHT), 5)
        
        for gate in self.gates:
            gate.draw()
        
        for lava in self.lava_pits:
            lava.draw()
        
        for enemy in self.enemies:
            enemy.draw()
        
        for bullet in self.bullets:
            bullet.draw()
        
        self.crowd.draw()
        
        # HUD
        count_box = pygame.Rect(10, 10, 180, 80)
        pygame.draw.rect(screen, DARK_GRAY, count_box)
        pygame.draw.rect(screen, CYAN, count_box, 3)
        count_label = small_font.render("CROWD", True, CYAN)
        screen.blit(count_label, (20, 15))
        count_text = big_font.render(f"{self.crowd.count}", True, WHITE)
        screen.blit(count_text, (25, 40))
        
        level_box = pygame.Rect(WIDTH - 190, 10, 180, 80)
        pygame.draw.rect(screen, DARK_GRAY, level_box)
        pygame.draw.rect(screen, YELLOW, level_box, 3)
        level_label = small_font.render("LEVEL", True, YELLOW)
        screen.blit(level_label, (WIDTH - 175, 15))
        level_text = big_font.render(f"{self.level}", True, WHITE)
        screen.blit(level_text, (WIDTH - 155, 40))
        
        gates_box = pygame.Rect(10, 100, 180, 60)
        pygame.draw.rect(screen, DARK_GRAY, gates_box)
        pygame.draw.rect(screen, GREEN, gates_box, 3)
        gates_label = small_font.render("GATES", True, GREEN)
        screen.blit(gates_label, (20, 105))
        gates_text = font.render(f"{self.gates_passed}/{self.gates_needed}", True, WHITE)
        screen.blit(gates_text, (25, 125))
        
        if self.game_over:
            # แสดง ai_dead.png เต็มจอก่อน
            if ai_dead_img:
                screen.blit(ai_dead_img, (0, 0))
            else:
                overlay = pygame.Surface((WIDTH, HEIGHT))
                overlay.set_alpha(200)
                overlay.fill(BLACK)
                screen.blit(overlay, (0, 0))
            
            # ซ้อนทับด้วย images.png ตรงกลาง
            if self.game_over_image:
                img_x = WIDTH // 2 - 200
                img_y = HEIGHT // 2 - 150
                screen.blit(self.game_over_image, (img_x, img_y))
            else:
                text = big_font.render("GAME OVER!", True, RED)
                screen.blit(text, (WIDTH//2 - 200, HEIGHT//2 - 50))
            
            restart_btn = Button(WIDTH//2 - 150, HEIGHT//2 + 200, 300, 80, "RESTART", GREEN)
            restart_btn.draw()
            return restart_btn
        
        if self.won:
            if ai_win_img:
                screen.blit(ai_win_img, (0, 0))
            else:
                overlay = pygame.Surface((WIDTH, HEIGHT))
                overlay.set_alpha(180)
                overlay.fill(BLACK)
                screen.blit(overlay, (0, 0))
            
            info_box = pygame.Surface((500, 250))
            info_box.set_alpha(180)
            info_box.fill(BLACK)
            screen.blit(info_box, (WIDTH//2 - 250, HEIGHT//2 - 125))
            
            text = big_font.render("LEVEL COMPLETE!", True, GREEN)
            screen.blit(text, (WIDTH//2 - 280, HEIGHT//2 - 100))
            
            score_text = font.render(f"Survivors: {self.crowd.count}", True, WHITE)
            screen.blit(score_text, (WIDTH//2 - 130, HEIGHT//2 - 20))
            
            if self.level < 3:
                next_text = small_font.render("Press N for Next Level", True, GREEN)
                screen.blit(next_text, (WIDTH//2 - 135, HEIGHT//2 + 40))
            else:
                win_text = font.render("YOU WIN THE GAME!", True, YELLOW)
                screen.blit(win_text, (WIDTH//2 - 200, HEIGHT//2 + 40))
            
            menu_text = small_font.render("Press M for Menu", True, WHITE)
            screen.blit(menu_text, (WIDTH//2 - 100, HEIGHT//2 + 80))


# ============================================
# ฟังก์ชันหน้าเมนู
# ============================================
def draw_menu():
    # ใช้ stage.png เป็น background
    if stage_bg:
        screen.blit(stage_bg, (0, 0))
    else:
        screen.fill(BLACK)
    
    # กล่องโปร่งแสงสำหรับ title
    title_box = pygame.Surface((700, 100))
    title_box.set_alpha(150)
    title_box.fill(BLACK)
    screen.blit(title_box, (WIDTH//2 - 350, 130))
    
    title = big_font.render("MATH CROWD RUNNER", True, CYAN)
    screen.blit(title, (WIDTH//2 - 320, 150))
    
    play_btn = Button(WIDTH//2 - 150, 300, 300, 100, "PLAY", GREEN)
    exit_btn = Button(WIDTH//2 - 150, 440, 300, 100, "EXIT", RED)
    
    play_btn.draw()
    exit_btn.draw()
    
    inst_font = pygame.font.Font(None, 24)
    inst1 = inst_font.render("Level 1: Basic Math (+, -, x)", True, WHITE)
    inst2 = inst_font.render("Level 2: Advanced (x, ÷, √, ^)", True, WHITE)
    inst3 = inst_font.render("Level 3: Extreme Difficulty", True, WHITE)
    
    screen.blit(inst1, (WIDTH//2 - 150, 560))
    screen.blit(inst2, (WIDTH//2 - 160, 590))
    screen.blit(inst3, (WIDTH//2 - 140, 620))
    
    return play_btn, exit_btn


# ============================================
# ลูปเกมหลัก
# ============================================
game = None
running = True

while running:
    mouse_pos = pygame.mouse.get_pos()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
        if event.type == pygame.MOUSEBUTTONDOWN:
            if game_state == "MENU":
                play_btn, exit_btn = draw_menu()
                if play_btn.check_click(mouse_pos):
                    game_state = "PLAYING"
                    current_level = 1
                    game = Game(current_level)
                    game.start_game()  # เริ่มเกมทันที
                    # เล่นเพลง
                    try:
                        pygame.mixer.music.play(-1)  # -1 = loop ไม่รู้จบ
                        pygame.mixer.music.set_volume(0.5)
                    except:
                        pass
                elif exit_btn.check_click(mouse_pos):
                    running = False
            
            elif game_state == "PLAYING" and game:
                if game.game_over:
                    restart_btn = game.draw()
                    if restart_btn and restart_btn.check_click(mouse_pos):
                        current_level = 1
                        game = Game(current_level)
                        game.start_game()  # เริ่มเกมทันที
                        # เล่นเพลงใหม่
                        try:
                            pygame.mixer.music.play(-1)
                            pygame.mixer.music.set_volume(0.5)
                        except:
                            pass
        
        if event.type == pygame.KEYDOWN:
            if game_state == "PLAYING" and game:
                if event.key == pygame.K_m:
                    game_state = "MENU"
                    game = None
                    # หยุดเพลงเมื่อกลับเมนู
                    try:
                        pygame.mixer.music.stop()
                    except:
                        pass
                elif event.key == pygame.K_n and game.won and current_level < 3:
                    current_level += 1
                    game = Game(current_level)
                    game.start_game()  # เริ่มด่านใหม่ทันที
    
    if game_state == "MENU":
        play_btn, exit_btn = draw_menu()
        play_btn.check_click(mouse_pos)
        exit_btn.check_click(mouse_pos)
        
    elif game_state == "PLAYING":
        if game:
            keys = pygame.key.get_pressed()
            if keys[pygame.K_LEFT] or keys[pygame.K_a]:
                game.crowd.move(-PLAYER_SPEED)
            if keys[pygame.K_RIGHT] or keys[pygame.K_d]:
                game.crowd.move(PLAYER_SPEED)
            
            game.update()
            restart_btn = game.draw()
            
            if restart_btn and game.game_over:
                restart_btn.check_click(mouse_pos)
    
    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-10-19 00:05:03.063 python[34069:6010105] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
